# Vamos a leer el cluster 3 y generar un dataset sencillo con NBEATS

vamos por paso

En este caso no voy a generar las covariables que generó Daiana

Voy a probar que corra, si corre luego le agrego las variables y veo los cambios en Kaggle




## Preparo todo

In [1]:
### me preparo en el entorno
# entorno
ColabNotebook = 'google.colab' in str(get_ipython())

if ColabNotebook:
    # monta G-drive en entorno COLAB
    from google.colab import drive
    drive.mount('/content/drive/')

    # carpeta donde se encuentran archivos .py auxiliares
    #FUENTES_DIR = '/content/drive/MyDrive/Colab Notebooks/FUENTES/'
    #DATOS_DIR = '/content/drive/MyDrive/Colab Notebooks/DATOS/'      # carpeta donde se encuentran los datasets
    FUENTES_DIR = '/content/drive/MyDrive/MCD/Labo3/FUENTES/' # Ajustar según usuario
    DATOS_DIR = '/content/drive/MyDrive/MCD_grupo17_la_desgracia/lab3/PruebasAK/DATOSL3/' # ajustar según usuario OJO crear un acceso directo para poder "montar" una unidad compartida
else:
    # configuración para notebook con instalación LOCAL
    # FUENTES_DIR = '../FUENTES'         # carpeta donde se encuentran archivos .py auxiliares
    DATOS_DIR   = './DATOS/' # carpeta donde se encuentran los datasets

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
## Cargo lo necesario
# Cargo paquetes
#Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from numpy.polynomial import Polynomial
#from scipy.signal import detrend



#import holidays

#Configuración de parametros de Pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.precision', 3)
pd.set_option('plotting.backend', 'matplotlib')

In [3]:
!pip install nbeats-pytorch

In [39]:
from sklearn.model_selection import train_test_split, TimeSeriesSplit, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin
import matplotlib.pyplot as plt
import torch
from nbeats_pytorch.model import NBeatsNet
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from torch.utils.data import DataLoader, TensorDataset




In [6]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 666.8 kB/s eta 0:00:00


In [7]:
import category_encoders as ce
import shap

In [8]:
# GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Usando dispositivo:', device)

Usando dispositivo: cuda


## Levanto el dataset
vamos a hacer el promedio de los últimos 12 meses por producto

In [9]:
# me traigo el archivo ya procesado con la duración completa
df_cluster_3 = pd.read_parquet(DATOS_DIR +'df_cluster_3.parquet')
df_cluster_3.head()
# productos a validar Son los que se van a estar vivos
productos_validar = pd.read_csv(DATOS_DIR +'product_id_apredecir201912.txt', sep='\t')
# me traigo el clasificador de clusters


In [10]:
df_cluster_3.shape

(174785, 45)

In [11]:
# me quedo con las filas de df_cluster_3 cuyo product_id esté en productos_validar
###################################################
##### OJO, en realidad debo dejar todo
##### dejo las otras series que ayuden
##### saco de todo el conjunto lo que necesito al final
#############################
#df_cluster_X = df_cluster_3[df_cluster_3['product_id'].isin(productos_validar['product_id'])]
df_cluster_X = df_cluster_3.copy()
df_cluster_X.shape

(174785, 45)

In [12]:
df_cluster_X.shape

(174785, 45)

In [13]:
df_cluster_X.dtypes


,0
customer_id,int64
product_id,int64
date_mes,period[M]
first_customer_date,period[M]
last_customer_date,period[M]
first_product_date,period[M]
last_product_date,period[M]
tn,float64
dif_tn,float64
dif_por,float64


In [14]:
# tengo que pasar a timestamp todas las columnas con formato period[M] de df_cluster_X
# Identificamos automáticamente las columnas de tipo Period[M]
period_cols = df_cluster_X.select_dtypes(include=['period[M]']).columns

# Convertimos todas esas columnas a datetime usando .dt.to_timestamp()
df_cluster_X[period_cols] = df_cluster_X[period_cols].apply(lambda col: col.dt.to_timestamp())
df_cluster_X.dtypes

,0
customer_id,int64
product_id,int64
date_mes,datetime64[ns]
first_customer_date,datetime64[ns]
last_customer_date,datetime64[ns]
first_product_date,datetime64[ns]
last_product_date,datetime64[ns]
tn,float64
dif_tn,float64
dif_por,float64


In [15]:
# 2. Preprocesamiento fechas usando 'date_mes' que es la marca temporal de tu dataset
# Aseguramos que 'date_mes' sea datetime y ordenamos
df_cluster_X['date_mes'] = pd.to_datetime(df_cluster_X['date_mes'])
df_cluster_X = df_cluster_X.sort_values(['product_cust_id', 'date_mes']).reset_index(drop=True)


## Ajustes varios dataset

#### known covariantes


## agrando el df con lo realizado por Daiana / CHAT

In [16]:
df_cluster_X.columns

Index(['customer_id', 'product_id', 'date_mes', 'first_customer_date',
       'last_customer_date', 'first_product_date', 'last_product_date', 'tn',
       'dif_tn', 'dif_por', 'vida_prod', 'vida_customer', 'meses_on',
       'meses_off', 'vida_restante', 'anio', 'mes', 'secuencia', 'trimestre',
       'cat1', 'cat1_cat2', 'cat1_cat2_cat3', 'cat1_cat2_cat3_brand',
       'cat1_cat2_cat3_brand_size', 'periodo', 'stock_final',
       'dias_laborables_arg', 'cisne_negro', 'VtaGral', 'VtaGralCte',
       'Bebidas', 'Almacén', 'Panadería', 'Lácteos', 'Carnes',
       'Verdulería y frutería', 'Artículos de limpieza y perfumería',
       'Indumentaria, calzado y textiles para el hogar',
       'Electrónicos y artículos para el hogar', 'Otros', 'USD_of', 'USD_blue',
       'Infl', 'Expect', 'product_cust_id'],
      dtype='object')

In [17]:
# 3. Media móvil de los últimos 3 meses
df_cluster_X['media_movil_3'] = df_cluster_X.groupby('product_cust_id')['tn'].transform(
    lambda x: x.shift(2).rolling(window=3, min_periods=1).mean()
)

In [18]:
# 3. Media móvil de los últimos 6 meses
df_cluster_X['media_movil_6'] = df_cluster_X.groupby('product_cust_id')['tn'].transform(
    lambda x: x.shift(2).rolling(window=6, min_periods=1).mean()
)

# 3. Media móvil de los últimos 9 meses
df_cluster_X['media_movil_9'] = df_cluster_X.groupby('product_cust_id')['tn'].transform(
    lambda x: x.shift(2).rolling(window=9, min_periods=1).mean()
)

# 3. Media móvil de los últimos 12 meses
df_cluster_X['media_movil_12'] = df_cluster_X.groupby('product_cust_id')['tn'].transform(
    lambda x: x.shift(2).rolling(window=12, min_periods=1).mean()
)

# Diferencia entre t-2 y t-3 (omitimos t y t-1)
df_cluster_X['delta_tn_1m'] = (
    df_cluster_X
    .groupby('product_cust_id')['tn']
    .transform(lambda x: x.shift(2) - x.shift(3))
)

In [19]:
# 5. Volatilidad (desvío estándar de los últimos 3 meses)
df_cluster_X['std_tn_3m'] = df_cluster_X.groupby('product_cust_id')['tn'].transform(
    lambda x: x.shift(2).rolling(window=3, min_periods=1).std()
)

# 5. Volatilidad (desvío estándar de los últimos 6 meses)
df_cluster_X['std_tn_6m'] = df_cluster_X.groupby('product_cust_id')['tn'].transform(
    lambda x: x.shift(2).rolling(window=6, min_periods=1).std()
)


# 7 Baja en t-2, t-3, t-4 (sin usar t ni t-1)
df_cluster_X['caida_reciente'] = (
    df_cluster_X
    .groupby('product_cust_id')['tn']
    .transform(lambda x: (x.shift(2) < x.shift(3)) & (x.shift(3) < x.shift(4)))
    .fillna(False)
    .astype(int)
)

# 8 Alza en t-2, t-3, t-4 (sin usar t ni t-1)
df_cluster_X['alza_reciente'] = (
    df_cluster_X
    .groupby('product_cust_id')['tn']
    .transform(lambda x: (x.shift(2) > x.shift(3)) & (x.shift(3) > x.shift(4)))
    .fillna(False)
    .astype(int)
)

In [20]:


# 9 Pendiente lineal (rolling trend)
def rolling_slope(x, window=4):
    return x.rolling(window).apply(lambda y: np.polyfit(range(len(y)), y, 1)[0], raw=False)

df_cluster_X['trend_4'] = df_cluster_X.groupby('product_cust_id')['tn'].transform(lambda x: rolling_slope(x.shift(2), window=4))

#10 mes con mayor venta historica
peak_month = df_cluster_X.groupby('product_cust_id')['tn'].idxmax().apply(lambda i: df_cluster_X.loc[i, 'date_mes'].month)
df_cluster_X = df_cluster_X.merge(peak_month.rename('mes_pico'), on='product_cust_id')
df_cluster_X['es_mes_pico'] = (df_cluster_X['date_mes'].dt.month == df_cluster_X['mes_pico']).astype(int)

# Variable: meses desde el máximo en los últimos 6 meses (sin contar t ni t+1)
def meses_desde_max_6m(grupo):
    resultados = []
    for i in range(len(grupo)):
        # Tomar los 6 meses anteriores a t-1 (es decir, desde t-2 a t-7)
        ventana = grupo.iloc[max(0, i - 7):max(0, i - 1)]
        if ventana.empty or ventana.isna().all():
            resultados.append(np.nan)
            continue
        max_valor = ventana.max()
        # Buscar el índice (posición) del último máximo dentro de la ventana
        idx_max = ventana[::-1].idxmax()
        meses_pasados = i - grupo.index.get_loc(idx_max)
        resultados.append(meses_pasados)
    return pd.Series(resultados, index=grupo.index)

df_cluster_X['meses_desde_max_6m'] = (
    df_cluster_X.groupby('product_cust_id')['tn']
    .apply(meses_desde_max_6m)
    .reset_index(level=0, drop=True)  # Alinear con df_cluster_X
)


In [21]:
# Venta total (por mes y cada nivel de categoría)
category_levels = [
    'cat1', 'cat1_cat2', 'cat1_cat2_cat3', 'cat1_cat2_cat3_brand', 'cat1_cat2_cat3_brand_size'
]

for cat in category_levels:
    # Total ventas por mes y categoria
    df_cluster_X[f'Vta_total_{cat}'] = df_cluster_X.groupby(['date_mes', cat])['tn'].transform('sum')
    # Rank dentro de categoría por mes (mayor tn = ranking 1)
    df_cluster_X[f'rank_{cat}'] = df_cluster_X.groupby(['date_mes', cat])['tn'].rank(method='first', ascending=False)
    # Participación (share)
    df_cluster_X[f'share_{cat}'] = df_cluster_X['tn'] / df_cluster_X[f'Vta_total_{cat}']

In [22]:
df_cluster_X.shape

(174785, 73)

## ahora normalizo

In [23]:
# --- Paso 1: Normalización robusta con clase para reusabilidad --- #
class RobustScalerCustom(BaseEstimator, TransformerMixin):
    """
    Escala series con std si std !=0, sino con media.
    Guarda parámetros para poder revertir la normalización.
    """
    def fit(self, X, y=None):
        self.mean_ = X.mean()
        self.std_ = X.std()
        if self.std_ == 0:
            self.std_ = None
            if self.mean_ == 0:
                self.mean_ = 1
        return self
    def transform(self, X):
        if self.std_ is not None:
            return (X - self.mean_) / self.std_
        else:
            return X / self.mean_
    def inverse_transform(self, X_scaled):
        if self.std_ is not None:
            return X_scaled * self.std_ + self.mean_
        else:
            return X_scaled * self.mean_

In [24]:
# lgbm no banca tildes ni espacios en los nombres de columna
# Diccionario de reemplazos para vocales con tilde
replacements = {
    'á': 'a',
    'é': 'e',
    'í': 'i',
    'ó': 'o',
    'ú': 'u',
    'Á': 'A',
    'É': 'E',
    'Í': 'I',
    'Ó': 'O',
    'Ú': 'U',
}

def clean_column_names(col_names):
    cleaned_cols = []
    for col in col_names:
        # Reemplazar espacios por guion bajo
        col = col.replace(' ', '_')
        # Eliminar comas
        col = col.replace(',', '')
        # Reemplazar vocales con tilde
        for orig, repl in replacements.items():
            col = col.replace(orig, repl)
        cleaned_cols.append(col)
    return cleaned_cols

# Aplica al DataFrame df_cluster_X
df_cluster_X.columns = clean_column_names(df_cluster_X.columns)

# Comprobar resultado
print(df_cluster_X.columns)

Index(['customer_id', 'product_id', 'date_mes', 'first_customer_date',
       'last_customer_date', 'first_product_date', 'last_product_date', 'tn',
       'dif_tn', 'dif_por', 'vida_prod', 'vida_customer', 'meses_on',
       'meses_off', 'vida_restante', 'anio', 'mes', 'secuencia', 'trimestre',
       'cat1', 'cat1_cat2', 'cat1_cat2_cat3', 'cat1_cat2_cat3_brand',
       'cat1_cat2_cat3_brand_size', 'periodo', 'stock_final',
       'dias_laborables_arg', 'cisne_negro', 'VtaGral', 'VtaGralCte',
       'Bebidas', 'Almacen', 'Panaderia', 'Lacteos', 'Carnes',
       'Verduleria_y_fruteria', 'Articulos_de_limpieza_y_perfumeria',
       'Indumentaria_calzado_y_textiles_para_el_hogar',
       'Electronicos_y_articulos_para_el_hogar', 'Otros', 'USD_of', 'USD_blue',
       'Infl', 'Expect', 'product_cust_id', 'media_movil_3', 'media_movil_6',
       'media_movil_9', 'media_movil_12', 'delta_tn_1m', 'std_tn_3m',
       'std_tn_6m', 'caida_reciente', 'alza_reciente', 'trend_4', 'mes_pico',
      

In [ ]:
# --- Paso 3: Selección columnas a normalizar y preparación de scalers --- #
#features_to_normalize = [
#    'tn','dif_tn','dif_por','vida_prod','vida_customer','meses_on','meses_off','vida_restante','stock_final',
#    'VtaGral','VtaGralCte','Bebidas','Almacen','Panaderia','Lacteos','Carnes','Verduleria_y_fruteria',
#    'Articulos_de_limpieza_y_perfumeria','Indumentaria_calzado_y_textiles_para_el_hogar',
#    'Electronicos_y_articulos_para_el_hogar','Otros','USD_of','USD_blue','Infl','Expect',
#    'media_movil_3','media_movil_6','media_movil_9','media_movil_12','delta_tn_1m',
#    'std_tn_3m','std_tn_6m','caida_reciente','alza_reciente','trend_4','mes_pico','es_mes_pico','meses_desde_max_6m','Vta_total_cat1', 'rank_cat1',
#    'share_cat1', 'Vta_total_cat1_cat2', 'rank_cat1_cat2',
#    'share_cat1_cat2', 'Vta_total_cat1_cat2_cat3', 'rank_cat1_cat2_cat3',
#    'share_cat1_cat2_cat3', 'Vta_total_cat1_cat2_cat3_brand',
#    'rank_cat1_cat2_cat3_brand', 'share_cat1_cat2_cat3_brand',
#    'Vta_total_cat1_cat2_cat3_brand_size', 'rank_cat1_cat2_cat3_brand_size',
#    'share_cat1_cat2_cat3_brand_size'
#]

# Creamos un diccionario para guardar scalers independientes por variable
#scalers_dict = {}

#for col in features_to_normalize:
#    scaler = RobustScalerCustom()
#    scaler.fit(df_cluster_X[col])
#    df_cluster_X[f'norm_{col}'] = scaler.transform(df_cluster_X[col])
#    scalers_dict[col] = scaler

In [25]:
# --- Paso 4: Preparar target a predecir: tn(t+2) --- #
df_cluster_X = df_cluster_X.sort_values(['product_cust_id', 'date_mes']).reset_index(drop=True)
df_cluster_X['tn_plus_2'] = df_cluster_X.groupby('product_cust_id')['tn'].shift(-2)
# Drop filas sin target (los dos últimos de cada serie)
df_cluster_X = df_cluster_X.dropna(subset=['tn_plus_2']).reset_index(drop=True)

# --- Paso 5: Normalizamos target usando scaler de 'tn' --- #
#como estoy normalizando abajo no necesito normalizar acá
#scaler_target = scalers_dict['tn']
#df_model['norm_tn_plus_2'] = scaler_target.transform(df_model['tn_plus_2'])
df_cluster_X.head()

,customer_id,product_id,date_mes,first_customer_date,last_customer_date,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,vida_customer,meses_on,meses_off,vida_restante,anio,mes,secuencia,trimestre,cat1,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size,periodo,stock_final,dias_laborables_arg,cisne_negro,VtaGral,VtaGralCte,Bebidas,Almacen,Panaderia,Lacteos,Carnes,Verduleria_y_fruteria,Articulos_de_limpieza_y_perfumeria,Indumentaria_calzado_y_textiles_para_el_hogar,Electronicos_y_articulos_para_el_hogar,Otros,USD_of,USD_blue,Infl,Expect,product_cust_id,media_movil_3,media_movil_6,media_movil_9,media_movil_12,delta_tn_1m,std_tn_3m,std_tn_6m,caida_reciente,alza_reciente,trend_4,mes_pico,es_mes_pico,meses_desde_max_6m,Vta_total_cat1,rank_cat1,share_cat1,Vta_total_cat1_cat2,rank_cat1_cat2,share_cat1_cat2,Vta_total_cat1_cat2_cat3,rank_cat1_cat2_cat3,share_cat1_cat2_cat3,Vta_total_cat1_cat2_cat3_brand,rank_cat1_cat2_cat3_brand,share_cat1_cat2_cat3_brand,Vta_total_cat1_cat2_cat3_brand_size,rank_cat1_cat2_cat3_brand_size,share_cat1_cat2_cat3_brand_size,tn_plus_2
0,10003,20001,2017-01-01,2017-01-01,2019-12-01,2017-01-01,2019-12-01,143.494,0.000,0.000e+00,4,4,4,0,60,2017,1,1,1,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201701,NaN,22,0,27858.87,27477.31,573064.06,1.624e+06,11576.02,253462.37,73883.84,20142.56,1.219e+06,1146.38,48728.12,269322.15,15.7,16.47,1.6,44.47,20001_10003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,8,0,NaN,3019.653,1.0,0.048,1407.105,1.0,0.102,603.549,1.0,0.238,322.418,1.0,0.445,290.268,1.0,0.494,137.875
1,10003,20001,2017-02-01,2017-01-01,2019-12-01,2017-01-01,2019-12-01,20.483,0.000,0.000e+00,5,5,5,0,60,2017,2,2,1,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201702,NaN,18,0,27858.87,27477.31,573064.06,1.624e+06,11576.02,253462.37,73883.84,20142.56,1.219e+06,1146.38,48728.12,269322.15,15.7,16.24,1.6,40.68,20001_10003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,8,0,NaN,5460.046,51.0,0.004,2371.082,27.0,0.009,587.224,7.0,0.035,175.027,3.0,0.117,128.283,3.0,0.160,68.893
2,10003,20001,2017-03-01,2017-01-01,2019-12-01,2017-01-01,2019-12-01,137.875,-0.056,-4.058e-04,6,6,6,0,60,2017,3,3,1,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201703,NaN,22,0,25994.20,25223.43,540122.06,1.559e+06,11395.77,252720.27,75545.85,20456.38,1.089e+06,1018.56,34128.35,257899.92,15.3,15.88,2.1,40.95,20001_10003,143.494,143.494,143.494,143.494,NaN,NaN,NaN,0,0,NaN,8,0,2.0,4852.938,1.0,0.028,2171.377,1.0,0.063,826.936,1.0,0.167,348.613,1.0,0.395,318.866,1.0,0.432,135.122
3,10003,20001,2017-04-01,2017-01-01,2019-12-01,2017-01-01,2019-12-01,68.893,-11.215,-1.400e-01,7,7,7,0,60,2017,4,4,2,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201704,NaN,18,0,27780.01,26334.26,586230.17,2.006e+06,12889.83,307951.54,89305.98,22607.84,1.296e+06,927.07,36321.56,301861.90,15.2,15.59,2.4,46.19,20001_10003,81.989,81.989,81.989,81.989,-123.011,86.982,86.982,0,0,NaN,8,0,3.0,4103.733,5.0,0.017,1717.161,3.0,0.040,579.903,3.0,0.119,322.808,3.0,0.213,285.444,3.0,0.241,171.018
4,10003,20001,2017-05-01,2017-01-01,2019-12-01,2017-01-01,2019-12-01,135.122,-14.999,-9.991e-02,8,8,8,0,60,2017,5,5,2,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201705,NaN,21,0,29134.27,27045.88,572661.71,1.967e+06,13642.25,298638.09,84722.68,24054.43,1.219e+06,1098.13,43054.66,298741.07,15.2,15.99,2.7,45.76,20001_10003,100.618,100.618,100.618,100.618,117.392,69.455,69.455,0,0,NaN,8,0,4.0,4849.908,3.0,0.028,2294.190,3.0,0.059,852.452,3.0,0.159,489.247,2.0,0.276,467.778,2.0,0.289,64.662


In [26]:
# 3. Normalización condicional (por columnas continuas que elijas)
def conditional_scaling(df, cols):
    scalers = {}
    for col in cols:
        vals = df[col].dropna()
        mean = vals.mean()
        std = vals.std()
        if std > 0:
            scaler = StandardScaler()
            scaler.fit(vals.values.reshape(-1,1))
            df[col] = scaler.transform(df[col].values.reshape(-1,1))
            scalers[col] = scaler
        else:
            df[col] = df[col].fillna(mean) - mean
            scalers[col] = None
    return df, scalers

cols_to_scale = [
  'tn', 'tn_plus_2', # acá va tn porque lo uso luego para calcular tn plus 2 y lo normalizo con lo que tengo
  'dif_tn', 'dif_por', 'vida_prod', 'vida_customer', 'meses_on', 'meses_off',
  'vida_restante', 'stock_final', 'VtaGral', 'VtaGralCte',
  'Bebidas','Almacen','Panaderia','Lacteos','Carnes','Verduleria_y_fruteria',
  'Articulos_de_limpieza_y_perfumeria','Indumentaria_calzado_y_textiles_para_el_hogar',
  'Electronicos_y_articulos_para_el_hogar','Otros','USD_of','USD_blue','Infl','Expect',
  'media_movil_3','media_movil_6','media_movil_9','media_movil_12','delta_tn_1m',
  'std_tn_3m','std_tn_6m','caida_reciente','alza_reciente','trend_4','mes_pico','es_mes_pico','meses_desde_max_6m','Vta_total_cat1', 'rank_cat1',
  'share_cat1', 'Vta_total_cat1_cat2', 'rank_cat1_cat2',
  'share_cat1_cat2', 'Vta_total_cat1_cat2_cat3', 'rank_cat1_cat2_cat3',
  'share_cat1_cat2_cat3', 'Vta_total_cat1_cat2_cat3_brand',
  'rank_cat1_cat2_cat3_brand', 'share_cat1_cat2_cat3_brand',
  'Vta_total_cat1_cat2_cat3_brand_size', 'rank_cat1_cat2_cat3_brand_size',
  'share_cat1_cat2_cat3_brand_size',
  'dias_laborables_arg', 'anio', 'mes', 'secuencia', 'trimestre', 'cisne_negro', 'periodo'
]

df_cluster_X, scalers = conditional_scaling(df_cluster_X, cols_to_scale)

In [27]:
df_cluster_X.columns

Index(['customer_id', 'product_id', 'date_mes', 'first_customer_date',
       'last_customer_date', 'first_product_date', 'last_product_date', 'tn',
       'dif_tn', 'dif_por', 'vida_prod', 'vida_customer', 'meses_on',
       'meses_off', 'vida_restante', 'anio', 'mes', 'secuencia', 'trimestre',
       'cat1', 'cat1_cat2', 'cat1_cat2_cat3', 'cat1_cat2_cat3_brand',
       'cat1_cat2_cat3_brand_size', 'periodo', 'stock_final',
       'dias_laborables_arg', 'cisne_negro', 'VtaGral', 'VtaGralCte',
       'Bebidas', 'Almacen', 'Panaderia', 'Lacteos', 'Carnes',
       'Verduleria_y_fruteria', 'Articulos_de_limpieza_y_perfumeria',
       'Indumentaria_calzado_y_textiles_para_el_hogar',
       'Electronicos_y_articulos_para_el_hogar', 'Otros', 'USD_of', 'USD_blue',
       'Infl', 'Expect', 'product_cust_id', 'media_movil_3', 'media_movil_6',
       'media_movil_9', 'media_movil_12', 'delta_tn_1m', 'std_tn_3m',
       'std_tn_6m', 'caida_reciente', 'alza_reciente', 'trend_4', 'mes_pico',
      

In [28]:
df_cluster_X.head()

,customer_id,product_id,date_mes,first_customer_date,last_customer_date,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,vida_customer,meses_on,meses_off,vida_restante,anio,mes,secuencia,trimestre,cat1,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size,periodo,stock_final,dias_laborables_arg,cisne_negro,VtaGral,VtaGralCte,Bebidas,Almacen,Panaderia,Lacteos,Carnes,Verduleria_y_fruteria,Articulos_de_limpieza_y_perfumeria,Indumentaria_calzado_y_textiles_para_el_hogar,Electronicos_y_articulos_para_el_hogar,Otros,USD_of,USD_blue,Infl,Expect,product_cust_id,media_movil_3,media_movil_6,media_movil_9,media_movil_12,delta_tn_1m,std_tn_3m,std_tn_6m,caida_reciente,alza_reciente,trend_4,mes_pico,es_mes_pico,meses_desde_max_6m,Vta_total_cat1,rank_cat1,share_cat1,Vta_total_cat1_cat2,rank_cat1_cat2,share_cat1_cat2,Vta_total_cat1_cat2_cat3,rank_cat1_cat2_cat3,share_cat1_cat2_cat3,Vta_total_cat1_cat2_cat3_brand,rank_cat1_cat2_cat3_brand,share_cat1_cat2_cat3_brand,Vta_total_cat1_cat2_cat3_brand_size,rank_cat1_cat2_cat3_brand_size,share_cat1_cat2_cat3_brand_size,tn_plus_2
0,10003,20001,2017-01-01,2017-01-01,2019-12-01,2017-01-01,2019-12-01,23.708,0.039,0.075,-1.406,-1.782,-0.263,-0.388,0.392,-1.337,-1.626,-1.861,-1.349,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,-1.409,NaN,1.115,-0.174,-1.284,0.658,-1.191,-1.495,-1.576,-1.414,-1.735,-1.145,-1.222,-0.984,-1.028,-1.579,-1.141,-1.098,-1.047,0.782,20001_10003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.296,-0.312,NaN,0.733,-0.313,NaN,0.377,-1.450,3.846,0.902,-1.196,5.648,0.625,-0.908,3.937,0.632,-0.999,6.150,2.468,-0.917,3.634,22.835
1,10003,20001,2017-02-01,2017-01-01,2019-12-01,2017-01-01,2019-12-01,3.187,0.039,0.075,-1.308,-1.681,-0.149,-0.388,0.392,-1.337,-1.321,-1.759,-1.349,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,-1.397,NaN,-1.802,-0.174,-1.284,0.658,-1.191,-1.495,-1.576,-1.414,-1.735,-1.145,-1.222,-0.984,-1.028,-1.579,-1.141,-1.113,-1.047,0.064,20001_10003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.296,-0.312,NaN,0.733,-0.313,NaN,2.069,-1.370,0.241,2.391,-1.095,0.320,0.584,-0.863,0.344,-0.050,-0.966,1.380,0.661,-0.840,0.889,11.298
2,10003,20001,2017-03-01,2017-01-01,2019-12-01,2017-01-01,2019-12-01,22.770,-0.052,0.064,-1.211,-1.581,-0.036,-0.388,0.392,-1.337,-1.016,-1.657,-1.349,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,-1.384,NaN,1.115,-0.174,-1.465,-0.266,-1.347,-1.573,-1.627,-1.420,-1.659,-1.092,-1.479,-1.231,-1.466,-1.716,-1.172,-1.136,-0.667,0.116,20001_10003,26.624,27.222,27.375,27.384,NaN,NaN,NaN,-0.296,-0.312,NaN,0.733,-0.313,-1.138e+00,1.648,-1.450,2.272,2.083,-1.196,3.451,1.182,-0.908,2.679,0.754,-0.999,5.430,2.787,-0.917,3.126,22.375
3,10003,20001,2017-04-01,2017-01-01,2019-12-01,2017-01-01,2019-12-01,11.263,-18.227,-3.892,-1.114,-1.480,0.078,-0.388,0.392,-1.337,-0.711,-1.555,-0.421,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,-1.372,NaN,-1.802,-0.174,-1.291,0.190,-1.129,-1.031,-1.204,-0.976,-1.032,-0.729,-1.071,-1.408,-1.400,-1.188,-1.179,-1.155,-0.440,1.107,20001_10003,15.099,15.436,15.521,15.525,-26.962,27.388,27.500,-0.296,-0.312,NaN,0.733,-0.313,-5.694e-01,1.128,-1.443,1.315,1.381,-1.188,2.117,0.566,-0.893,1.830,0.634,-0.966,2.782,2.414,-0.840,1.559,28.379
4,10003,20001,2017-05-01,2017-01-01,2019-12-01,2017-01-01,2019-12-01,22.311,-24.388,-2.756,-1.017,-1.380,0.191,-0.388,0.392,-1.337,-0.406,-1.453,-0.421,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,-1.359,NaN,0.386,-0.174,-1.159,0.481,-1.193,-1.077,-0.992,-1.051,-1.241,-0.485,-1.223,-1.077,-1.198,-1.226,-1.179,-1.129,-0.212,1.026,20001_10003,18.590,19.006,19.112,19.117,25.734,21.813,21.895,-0.296,-0.312,NaN,0.733,-0.313,-6.457e-04,1.646,-1.447,2.227,2.272,-1.188,3.189,1.246,-0.893,2.533,1.404,-0.982,3.694,4

In [ ]:
# Columnas estáticas, categóricas o numéricas (ajustar al caso real):
#static_cols = [
#    "customer_id","product_id","cat1","cat1_cat2","cat1_cat2_cat3",
#    "cat1_cat2_cat3_brand","cat1_cat2_cat3_brand_size"
#]
#extern_cols = [
#    "dias_laborables_arg","anio","mes","secuencia","trimestre","cisne_negro","periodo"
#]

In [30]:
# 4. Encoding variables estáticas categóricas (producto, cliente, categorías)
static_cat_cols = ['customer_id','product_id','cat1', 'cat1_cat2', 'cat1_cat2_cat3',
                   'cat1_cat2_cat3_brand', 'cat1_cat2_cat3_brand_size']

encoder = ce.TargetEncoder()
df_cluster_X[static_cat_cols] = encoder.fit_transform(df_cluster_X[static_cat_cols], df_cluster_X['tn_plus_2'])

In [31]:
df_cluster_X.head()

,customer_id,product_id,date_mes,first_customer_date,last_customer_date,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,vida_customer,meses_on,meses_off,vida_restante,anio,mes,secuencia,trimestre,cat1,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size,periodo,stock_final,dias_laborables_arg,cisne_negro,VtaGral,VtaGralCte,Bebidas,Almacen,Panaderia,Lacteos,Carnes,Verduleria_y_fruteria,Articulos_de_limpieza_y_perfumeria,Indumentaria_calzado_y_textiles_para_el_hogar,Electronicos_y_articulos_para_el_hogar,Otros,USD_of,USD_blue,Infl,Expect,product_cust_id,media_movil_3,media_movil_6,media_movil_9,media_movil_12,delta_tn_1m,std_tn_3m,std_tn_6m,caida_reciente,alza_reciente,trend_4,mes_pico,es_mes_pico,meses_desde_max_6m,Vta_total_cat1,rank_cat1,share_cat1,Vta_total_cat1_cat2,rank_cat1_cat2,share_cat1_cat2,Vta_total_cat1_cat2_cat3,rank_cat1_cat2_cat3,share_cat1_cat2_cat3,Vta_total_cat1_cat2_cat3_brand,rank_cat1_cat2_cat3_brand,share_cat1_cat2_cat3_brand,Vta_total_cat1_cat2_cat3_brand_size,rank_cat1_cat2_cat3_brand_size,share_cat1_cat2_cat3_brand_size,tn_plus_2
0,10003,20001,2017-01-01,2017-01-01,2019-12-01,2017-01-01,2019-12-01,23.708,0.039,0.075,-1.406,-1.782,-0.263,-0.388,0.392,-1.337,-1.626,-1.861,-1.349,0.076,0.154,0.344,0.817,2.59,-1.409,NaN,1.115,-0.174,-1.284,0.658,-1.191,-1.495,-1.576,-1.414,-1.735,-1.145,-1.222,-0.984,-1.028,-1.579,-1.141,-1.098,-1.047,0.782,20001_10003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.296,-0.312,NaN,0.733,-0.313,NaN,0.377,-1.450,3.846,0.902,-1.196,5.648,0.625,-0.908,3.937,0.632,-0.999,6.150,2.468,-0.917,3.634,22.835
1,10003,20001,2017-02-01,2017-01-01,2019-12-01,2017-01-01,2019-12-01,3.187,0.039,0.075,-1.308,-1.681,-0.149,-0.388,0.392,-1.337,-1.321,-1.759,-1.349,0.076,0.154,0.344,0.817,2.59,-1.397,NaN,-1.802,-0.174,-1.284,0.658,-1.191,-1.495,-1.576,-1.414,-1.735,-1.145,-1.222,-0.984,-1.028,-1.579,-1.141,-1.113,-1.047,0.064,20001_10003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.296,-0.312,NaN,0.733,-0.313,NaN,2.069,-1.370,0.241,2.391,-1.095,0.320,0.584,-0.863,0.344,-0.050,-0.966,1.380,0.661,-0.840,0.889,11.298
2,10003,20001,2017-03-01,2017-01-01,2019-12-01,2017-01-01,2019-12-01,22.770,-0.052,0.064,-1.211,-1.581,-0.036,-0.388,0.392,-1.337,-1.016,-1.657,-1.349,0.076,0.154,0.344,0.817,2.59,-1.384,NaN,1.115,-0.174,-1.465,-0.266,-1.347,-1.573,-1.627,-1.420,-1.659,-1.092,-1.479,-1.231,-1.466,-1.716,-1.172,-1.136,-0.667,0.116,20001_10003,26.624,27.222,27.375,27.384,NaN,NaN,NaN,-0.296,-0.312,NaN,0.733,-0.313,-1.138e+00,1.648,-1.450,2.272,2.083,-1.196,3.451,1.182,-0.908,2.679,0.754,-0.999,5.430,2.787,-0.917,3.126,22.375
3,10003,20001,2017-04-01,2017-01-01,2019-12-01,2017-01-01,2019-12-01,11.263,-18.227,-3.892,-1.114,-1.480,0.078,-0.388,0.392,-1.337,-0.711,-1.555,-0.421,0.076,0.154,0.344,0.817,2.59,-1.372,NaN,-1.802,-0.174,-1.291,0.190,-1.129,-1.031,-1.204,-0.976,-1.032,-0.729,-1.071,-1.408,-1.400,-1.188,-1.179,-1.155,-0.440,1.107,20001_10003,15.099,15.436,15.521,15.525,-26.962,27.388,27.500,-0.296,-0.312,NaN,0.733,-0.313,-5.694e-01,1.128,-1.443,1.315,1.381,-1.188,2.117,0.566,-0.893,1.830,0.634,-0.966,2.782,2.414,-0.840,1.559,28.379
4,10003,20001,2017-05-01,2017-01-01,2019-12-01,2017-01-01,2019-12-01,22.311,-24.388,-2.756,-1.017,-1.380,0.191,-0.388,0.392,-1.337,-0.406,-1.453,-0.421,0.076,0.154,0.344,0.817,2.59,-1.359,NaN,0.386,-0.174,-1.159,0.481,-1.193,-1.077,-0.992,-1.051,-1.241,-0.485,-1.223,-1.077,-1.198,-1.226,-1.179,-1.129,-0.212,1.026,20001_10003,18.590,19.006,19.112,19.117,25.734,21.813,21.895,-0.296,-0.312,NaN,0.733,-0.313,-6.457e-04,1.646,-1.447,2.227,2.272,-1.188,3.189,1.246,-0.893,2.533,1.404,-0.982,3.694,4.449,-0.878,1.949,10.590


In [32]:
# 5. Definición variables para modelado

history_size = 6  # meses atrás dejo 6 porque es lo que filtré antes
horizon = 2        # predecimos 2 meses adelante (feb2020)

features_time = [
  'tn',
  'dif_tn', 'dif_por', 'vida_prod', 'vida_customer', 'meses_on',
  'meses_off', 'vida_restante', 'anio', 'mes', 'secuencia', 'trimestre',
  'periodo', 'stock_final',
  'dias_laborables_arg', 'cisne_negro', 'VtaGral', 'VtaGralCte',
  'Bebidas', 'Almacen', 'Panaderia', 'Lacteos', 'Carnes',
  'Verduleria_y_fruteria', 'Articulos_de_limpieza_y_perfumeria',
  'Indumentaria_calzado_y_textiles_para_el_hogar',
  'Electronicos_y_articulos_para_el_hogar', 'Otros', 'USD_of', 'USD_blue',
  'Infl', 'Expect', 'product_cust_id', 'media_movil_3', 'media_movil_6',
  'media_movil_9', 'media_movil_12', 'delta_tn_1m', 'std_tn_3m',
  'std_tn_6m', 'caida_reciente', 'alza_reciente', 'trend_4', 'mes_pico',
  'es_mes_pico', 'meses_desde_max_6m', 'Vta_total_cat1', 'rank_cat1',
  'share_cat1', 'Vta_total_cat1_cat2', 'rank_cat1_cat2',
  'share_cat1_cat2', 'Vta_total_cat1_cat2_cat3', 'rank_cat1_cat2_cat3',
  'share_cat1_cat2_cat3', 'Vta_total_cat1_cat2_cat3_brand',
  'rank_cat1_cat2_cat3_brand', 'share_cat1_cat2_cat3_brand',
  'Vta_total_cat1_cat2_cat3_brand_size', 'rank_cat1_cat2_cat3_brand_size',
  'share_cat1_cat2_cat3_brand_size', 'tn_plus_2'
]

features_static = static_cat_cols

In [43]:
def create_samples(df, feature_time_cols, feature_static_cols, target_col, history_size, horizon): ### esto no me sirve, porque al muestrear me quedo sin datos.
    X_samples = []
    y_samples = []

    feat_static = df[feature_static_cols].iloc[0].values.astype(np.float32)  # <-- añadido .astype(np.float32)

    df_dyn = df.sort_values('date_mes')
    feat_dyn = df_dyn[feature_time_cols].values.astype(np.float32)  # <-- añadido .astype(np.float32)
    target = df_dyn[target_col].values.astype(np.float32)           # <-- añadido .astype(np.float32)

    for i in range(len(df) - history_size - horizon + 1):
        window_dyn = feat_dyn[i:i + history_size].flatten()
        x = np.concatenate([window_dyn, feat_static])
        y = target[i + history_size:i + history_size + horizon]
        if np.any(np.isnan(y)) or np.any(np.isnan(x)):
            continue  # evitar muestras con valores nan (por seguridad)
        X_samples.append(x)
        y_samples.append(y)

    return np.array(X_samples), np.array(y_samples)

In [53]:
horizon

2

In [57]:
def series_as_single_sample(df, features_time, features_static, target_col, horizon):
    """
    Prepara toda la serie como UNA muestra:
    X = toda la serie menos últimos horizon pasos,
    Y = target de los últimos 'horizon' pasos.
    """
    # Asegúrate de ordenar
    df = df.sort_values('date_mes')
    feat_static = df[features_static].iloc[0].values.astype(np.float32)

    # Todas las columnas dinámicas salvo los últimos horizon puntos
    feat_dyn = df[features_time].iloc[:-horizon].values.flatten().astype(np.float32)
    x = np.concatenate([feat_dyn, feat_static])

    # Target: los últimos "horizon" pasos
    y = df[target_col].iloc[-horizon:].values.astype(np.float32)
    return x, y

In [58]:
# para cada grupo serie
X_all = []
y_all = []
for prod_cust_id, df_group in df_cluster_X.groupby(['product_cust_id']):
    # Solo procesar series que tengan al menos horizon+1 pasos
    if len(df_group) > horizon:
        x, y = series_as_single_sample(df_group, features_time, features_static, 'tn_plus_2', horizon)
        X_all.append(x)
        y_all.append(y)

X = np.stack(X_all)
y = np.stack(y_all)

ValueError: all input arrays must have the same shape

In [35]:
from nbeats_pytorch.model import NBeatsNet
help(NBeatsNet.__init__)

Help on function __init__ in module nbeats_pytorch.model:

__init__(self, device=device(type='cpu'), stack_types=('trend', 'seasonality'), nb_blocks_per_stack=3, forecast_length=5, backcast_length=10, thetas_dim=(4, 8), share_weights_in_stack=False, hidden_layer_units=256, nb_harmonics=None)
    Initialize internal Module state, shared by both nn.Module and ScriptModule.



In [38]:
# 7. Definición del modelo N-BEATS
input_dim = history_size * len(features_time) + len(features_static)
forecast_length = horizon
model = NBeatsNet(
    device=device,  # calcula dinámicamente si tienes GPU
    stack_types=('trend', 'seasonality'),
    nb_blocks_per_stack=2,
    forecast_length=2,       # predices dos meses adelante
    backcast_length=6,      # usas 30 meses de historia
    thetas_dim=(4, 8),       # típicos valores
    share_weights_in_stack=False,
    hidden_layer_units=256   # puedes aumentar si tu hardware lo permite, pero 128 es razonable
    # nb_harmonics=None      # deja el default
)

| N-Beats
| --  Stack Trend (#0) (share_weights_in_stack=False)
     | -- TrendBlock(units=256, thetas_dim=4, backcast_length=6, forecast_length=2, share_thetas=True) at @139047986968464
     | -- TrendBlock(units=256, thetas_dim=4, backcast_length=6, forecast_length=2, share_thetas=True) at @139047986960080
| --  Stack Seasonality (#1) (share_weights_in_stack=False)
     | -- SeasonalityBlock(units=256, thetas_dim=2, backcast_length=6, forecast_length=2, share_thetas=True) at @139047702682128
     | -- SeasonalityBlock(units=256, thetas_dim=2, backcast_length=6, forecast_length=2, share_thetas=True) at @139047986972944


In [40]:
epochs = 30
batch_size = 64
learning_rate = 1e-3

criterion = torch.nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

predicciones_finales = {}

In [48]:
print(history_size)

6


In [52]:
print("Comienzo entrenamiento y pronóstico...")

for prod_cust, df_group in df_cluster_X.groupby('product_cust_id'):
    df_group = df_group.reset_index(drop=True)

    # Crear muestras para entrenamiento (solo filas donde target existe)
    X, y = create_samples(df_group, features_time, features_static, 'tn_plus_2', history_size, horizon)
    if len(X) < 7:
        print(f"Serie {prod_cust} muy corta para entrenar, se salta.")
        continue

    X = torch.tensor(X, dtype=torch.float32).to(device)
    y = torch.tensor(y, dtype=torch.float32).to(device)
    print(f"Tamaño de X: {X.shape}, Tamaño de y: {y.shape}")

    dataset = TensorDataset(X, y)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Entrenamiento
    for epoch in range(epochs):
        model.train()
        loss_total = 0
        for xx, yy in loader:
            optimizer.zero_grad()
            preds = model(xx)
            loss = criterion(preds, yy)
            loss.backward()
            optimizer.step()
            loss_total += loss.item()
        if (epoch+1)%10==0:
            print(f'{prod_cust} - Epoch {epoch+1}/{epochs} Loss: {loss_total/len(loader):.4f}')

Se truncaron las últimas líneas 5000 del resultado de transmisión.
Serie 20055_10087 muy corta para entrenar, se salta.
Serie 20055_10089 muy corta para entrenar, se salta.
Serie 20055_10099 muy corta para entrenar, se salta.
Serie 20055_10112 muy corta para entrenar, se salta.
Serie 20055_10122 muy corta para entrenar, se salta.
Serie 20055_10153 muy corta para entrenar, se salta.
Serie 20056_10011 muy corta para entrenar, se salta.
Serie 20056_10030 muy corta para entrenar, se salta.
Serie 20056_10046 muy corta para entrenar, se salta.
Serie 20056_10069 muy corta para entrenar, se salta.
Serie 20056_10098 muy corta para entrenar, se salta.
Serie 20056_10121 muy corta para entrenar, se salta.
Serie 20056_10132 muy corta para entrenar, se salta.
Serie 20057_10001 muy corta para entrenar, se salta.
Serie 20057_10004 muy corta para entrenar, se salta.
Serie 20057_10006 muy corta para entrenar, se salta.
Serie 20057_10017 muy corta para entrenar, se salta.
Serie 20057_10019 muy corta para

In [51]:
    # Predicción para 2020-02: usar últimos history_size meses con features dinámicas y estáticas
    df_last = df_group.sort_values('date_mes').iloc[-history_size:]
    x_dyn = df_last[features_time].values.flatten()
    x_static = df_last[features_static].iloc[0].values
    x_input = np.concatenate([x_dyn, x_static])
    x_tensor = torch.tensor(x_input.astype(np.float32)).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        y_pred_norm = model(x_tensor).cpu().numpy().flatten()

    # Desnormalizamos el target
    scaler_target = scalers.get('norm_tn_plus_2')
    if scaler_target:
        y_pred = scaler_target.inverse_transform(y_pred_norm.reshape(-1,1)).flatten()
    else:
        y_pred = y_pred_norm

    predicciones_finales[prod_cust] = y_pred
    print(f"Predicción 2 meses adelante para {prod_cust}: {y_pred}")

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x317 and 6x256)

In [ ]:
param_dist = {
    'num_leaves': [31, 50, 70, 100],
    'max_depth': [5, 10, 20, -1],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [50, 100, 200],
    'min_child_samples': [10, 20, 30],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
}

lgb_estimator = lgb.LGBMRegressor(random_state=42)

tscv = TimeSeriesSplit(n_splits=3)

In [ ]:
random_search = RandomizedSearchCV(
    estimator=lgb_estimator,
    param_distributions=param_dist,
    n_iter=30,
    cv=tscv,
    scoring='neg_root_mean_squared_error',
    verbose=1,
    random_state=42,
    n_jobs=-1,
)

random_search.fit(X_train, y_train)

print("Mejores parametros:", random_search.best_params_)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


In [ ]:
# --- Paso 9: Modelo final con mejores parametros --- #
best_params = random_search.best_params_
model = lgb.LGBMRegressor(**best_params, random_state=42)
model.fit(X_train, y_train)

In [ ]:
# Predicción y desfase inverso para interpretar salida ---
y_val_pred = model.predict(X_val)
# Transformamos predicciones de vuelta a escala original
y_val_pred_original = scaler_target.inverse_transform(y_val_pred)
y_val_original = scaler_target.inverse_transform(y_val.values)

rmse_val = mean_squared_error(y_val_original, y_val_pred_original, squared=False)
print(f"RMSE validación (escala original): {rmse_val:.4f}")

In [ ]:
# --- Paso 10: Forecast a futuro para el último dato de cada product_cust_id --- #
# Tomar los últimos registros de cada serie para predecir tn+2
to_forecast = df_cluster_X.groupby('product_cust_id').tail(1)  # último punto (de algún modo, adaptar según sea necesario)

# Preprocesar features igual que antes
for col in static_cols + extern_cols:
    if col in to_forecast.columns and to_forecast[col].dtype == 'object':
        to_forecast[col] = to_forecast[col].astype('category').cat.codes
for col in features_to_normalize:
    col_norm = f'norm_{col}'
    if col_norm not in to_forecast.columns:
        to_forecast[col_norm] = scalers_dict[col].transform(to_forecast[col])

X_forecast = to_forecast[feature_cols]
forecast_norm = model.predict(X_forecast)
forecast = scaler_target.inverse_transform(forecast_norm)

resultados = to_forecast[['product_cust_id', 'date_mes']].copy()
resultados['forecast_tn_plus_2'] = forecast

print(resultados.head())

### maniobrando para volver

In [ ]:
df_pred = pd.DataFrame(predictions_febrero_2020['mean'])
df_pred = df_pred.reset_index()
df_pred.head()

,item_id,timestamp,mean
0,20003_10001,2020-02-29,103.520
1,20004_10001,2020-02-29,19.771
2,20006_10001,2020-02-29,13.402
3,20008_10001,2020-02-29,18.388
4,20011_10001,2020-02-29,33.734


## manioboro para sacar la predicción

In [ ]:
# creo la columna product_id como los cinco primeros caracteres de la columna item_id de df_pred
df_pred['product_id'] = df_pred['item_id'].str[:5]
df_pred.head()

,item_id,timestamp,mean,product_id
0,20003_10001,2020-02-29,103.520,20003
1,20004_10001,2020-02-29,19.771,20004
2,20006_10001,2020-02-29,13.402,20006
3,20008_10001,2020-02-29,18.388,20008
4,20011_10001,2020-02-29,33.734,20011


In [ ]:
# obtengo df_pred1 como la suma de mean para cada timestamp y cada product_id
df_pred1 = df_pred.groupby(['product_id', 'timestamp'])['mean'].sum().reset_index()
df_pred1.head()

,product_id,timestamp,mean
0,20001,2020-02-29,335.919
1,20002,2020-02-29,160.225
2,20003,2020-02-29,227.226
3,20004,2020-02-29,82.912
4,20005,2020-02-29,103.366


In [ ]:
# llevo la columna timestamp a formato period[M]
df_pred1['timestamp'] = pd.to_datetime(df_pred1['timestamp'])
df_pred1['timestamp'] = df_pred1['timestamp'].dt.to_period('M')
df_pred1.head()

,product_id,timestamp,mean
0,20001,2020-02,335.919
1,20002,2020-02,160.225
2,20003,2020-02,227.226
3,20004,2020-02,82.912
4,20005,2020-02,103.366


In [ ]:
df_pred1.shape

(779, 3)

In [ ]:
df_pred1.dtypes

,0
product_id,object
timestamp,period[M]
mean,float64


In [ ]:
#llevo la columna product_id de df_pred1 a integrer
df_pred1['product_id'] = df_pred1['product_id'].astype(int)
df_pred1.dtypes

,0
product_id,int64
timestamp,period[M]
mean,float64


In [ ]:
productos_validar.head()

,product_id
0,20001
1,20002
2,20003
3,20004
4,20005


In [ ]:
productos_validar.shape

(780, 1)

In [ ]:
productos_validar.dtypes

,0
product_id,int64


In [ ]:
# Merge para obtener el promedio solo para los product_id que están en productos_validar
df_validar = productos_validar.merge(
    df_pred1,
    on='product_id',
    how='left'
)

# Ahora df_validar_promedio tendrá dos columnas: product_id y tn_prom_2018
print(df_validar.head())

   product_id timestamp     mean
0       20001   2020-02  335.919
1       20002   2020-02  160.225
2       20003   2020-02  227.226
3       20004   2020-02   82.912
4       20005   2020-02  103.366


In [ ]:
df_validar.tail()

,product_id,timestamp,mean
775,21263,NaT,NaN
776,21265,NaT,NaN
777,21266,NaT,NaN
778,21267,NaT,NaN
779,21276,NaT,NaN


In [ ]:
df_validar.shape

(780, 3)

In [ ]:
# llevo al formato deseado, dropeo timestamp
df_validar_promedio = df_validar.drop(columns=['timestamp'])
df_validar_promedio.head()

,product_id,mean
0,20001,335.919
1,20002,160.225
2,20003,227.226
3,20004,82.912
4,20005,103.366


In [ ]:
# le cambio el nombre de mean a tn
df_validar_promedio.rename(columns={'mean': 'tn'}, inplace=True)
df_validar_promedio.head()

,product_id,tn
0,20001,335.919
1,20002,160.225
2,20003,227.226
3,20004,82.912
4,20005,103.366


In [ ]:
df_validar_promedio.to_csv(DATOS_DIR +'df_cluster3_auto_V2.csv', index=False)